In [5]:
# ✅ Set Gemini API key securely (Google Colab specific)
import os

from dotenv import load_dotenv

load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

from app.prompts import LEARNING_PLANNER, LEARNING_RESEARCHER
from app.schema import LearningPlan

In [3]:

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5, max_retries=2, api_key=gemini_api_key)
struct_bound = llm.with_structured_output(LearningPlan)

system_message = LEARNING_PLANNER.format(plan=None)

response = struct_bound.invoke([("system", system_message), ("user", "how to be a pro drummer")])

print(response)


## research agent

response = llm.invoke([("system", LEARNING_RESEARCHER), ("user", "how to be a pro drummer")])

print(response)



topic='Pro Drumming' duration_weeks=4 weekly_plans=[WeekPlan(week_number=1, focus='Establish Fundamentals and Ergonomics', activities=[Activity(description='Set up your kit ergonomically', frequency='Daily'), Activity(description='Practice basic rudiments (single stroke, double stroke, paradiddle)', frequency='Daily'), Activity(description='Listen to recordings of your favorite drummers', frequency='Daily'), Activity(description='Work on grip technique', frequency='3x/week')], resources=[Resource(name='Stick Control for the Snare Drummer', type='book', url=None), Resource(name="The Drummer's Complete Vocabulary", type='book', url=None), Resource(name='Online drumming tutorials', type='other', url='https://www.example.com/drum-tutorials')], checkpoint='Record yourself playing a basic beat with good time and feel.'), WeekPlan(week_number=2, focus='Develop Groove and Independence', activities=[Activity(description='Practice independence exercises (e.g., bass drum on 1 & 3, snare on 2 & 4,

In [4]:
# ✅ 🔧 MISSING FUNCTION: generate_learning_path()
def generate_learning_path(topic):
    response = struct_bound.invoke([("system", system_message), ("user", topic)])
    return response.model_dump_json(indent=2)


# ✅ Optional test to see if Gemini works (can comment out later)
print(generate_learning_path("Python"))

{
  "topic": "Python",
  "duration_weeks": 4,
  "weekly_plans": [
    {
      "week_number": 1,
      "focus": "Understanding basic Python syntax and data types",
      "activities": [
        {
          "description": "Read introductory chapter on Python syntax",
          "frequency": "Once"
        },
        {
          "description": "Complete basic Python tutorial on data types",
          "frequency": "Daily"
        },
        {
          "description": "Practice writing simple programs using variables and operators",
          "frequency": "3 times a week"
        }
      ],
      "resources": [
        {
          "name": "Python Crash Course",
          "type": "book",
          "url": null
        },
        {
          "name": "Codecademy Python Course",
          "type": "app",
          "url": "https://www.codecademy.com/learn/learn-python-3"
        }
      ],
      "checkpoint": "Submit a program that calculates the area of a rectangle"
    },
    {
      "week_number

In [ ]:
# ✅ Gradio interface
import gradio as gr


def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)


def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename


with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch(share=True, debug=True)